In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
#Hyperparameters
learning_rate = 3E-6
adam_epsilon = 1E-8
number_of_epochs = 15
warmup = 3
mini_batch_size = 32

In [3]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('I will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


C:\Users\User\AppData\Local\Temp\ipykernel_14960\3106764981.py:8: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  device = torch.device("cpu")
